#### AMPEL intro

AMPEL is software framework designed for processing heterogeneous streamed data. 

AMPEL was not developed to provide a specific scientific resource, but rather an environment where it is easy to ensure that a scientific program fulfills the strict requirement of the next generation real-time experiments: efficient and powerful analysis, where provenance and reproducibiltiy is paramount. In particular, to guarantee the last point requires algorithms (which make real-time deicsions) be separated from infrastructure (which will likely evolve with time and project phase).

An AMPEL _user_ constructs a configuration file which describes every step of how an incoming alert stream should be processed. This can be broken down into selecting which _units_ should be executed, and which _parameters_ each of these should be provided. An AMPEL _live instance_ executes these units, based on the input data, as requested and stores all intermediate and final data in a databse. 

Provenance/reproducibility is ensured through multiple layers. First, each live instance is run from a container which can be retrieved later and together with a data archive replay the full stream. Second, AMPEL contains an extensive set of logs and a transient-specific _Journal_ which details all related actions/decisions. Finally, each unit and channel configuration file is drawn from a specific (tagged) github version. 

The series of notebooks provided here gradually builds toward a sample full configration.

#### Sample science case

Each AMPEl _channel_ is designed with a science goal (or "hypothesis/test") in mind. A much discussed current topic is the origin of the extragalactic neutrino flux observed e.g. by IceCube, with one of the potential sources being supernovae interacting with circumstellar material (SNIIn). We here wish to investigate whether a particular subtype of these, SN2009ip-like SNe with recent previous outbursts, are regularly found within the uncertainty region of neutrino alerts. 

The steps for this science program would be: Identify transients with optical lightcurves compatible with SN2009ip AND which coincide with neutrino alerts. For such targets, obtain follow-up spectroscopy to confirm classification (i.e. an external reaction). 

#### This notebook - Tutorial 4

This notebook reproduces the results of Tutorial 3, but through processing the `SampleChannel` configuration found at

`Ampel-contrib-sample/conf/ampel-contrib-sample/channel/SAMPLE_CHANNEL.yml` 

At this stage the channel is ready to be included in a live AMPEL instance. This can either be used to process a large set of archive data, or for processing a real-time data stream. Simultaneously, the channel configuration and the unit algorithms serves to provide a full, referencable description of the science content of the channel. 


As in Tutorial 2, this notebook thus assumes a mongod instance to be running and accessible through 27017. (The port can be changed through the mongo key of the ampel_config.yml file).

In [ ]:
import os
%load_ext ampel_quick_import
%qi DevAmpelContext AmpelLogger T2Processor T3Processor ChannelModel AlertProcessor TarAlertLoader ChannelModel AbsAlertFilter ProcessModel DefaultProcessController

In [ ]:
AMPEL_CONF = "../../ampel_config.yml"
ALERT_ARCHIVE = '../sample_data/ztfpub_200917_pruned.tar.gz'

In [ ]:
# The operation context is created based on a setup configuration file.
# db_prefix sets the DB name to use
ctx = DevAmpelContext.load(
    config_file_path = AMPEL_CONF,
    db_prefix = "AmpelTutorial",
    purge_db = True,
)

In [ ]:
ctx.config.get( "process.t0.SAMPLE_CHANNEL|T0|ztf_uw_public" )

In [ ]:
pm = ProcessModel(**ctx.config.get( "process.t0.SAMPLE_CHANNEL|T0|ztf_uw_public" ))

In [ ]:
DefaultProcessController??

In [ ]:
controller = DefaultProcessController(config=ctx.config, processes=[pm])

In [ ]:
controller.run()

In [ ]:
controller.run_process(pm=pm)

In [ ]:
pmt2 = ProcessModel(**ctx.config.get( "process.t2.DefaultT2Process" ))

In [ ]:
controller.run_process(pm=pmt2)

In [ ]:
pmt3 = ProcessModel(**ctx.config.get( "process.t3.SAMPLE_CHANNEL|T3|summary_00" ))

In [ ]:
controller.run_process(pm=pmt3)

So far an input data stream has been filtered, and some sort of calculation has been done on the accepted sample. The next step for a full channel is usually some sort of _reaction_. These can vary between sending immediate alarms (e.g. through Slack or GCN), triggering follow-up observations or propagating information (e.g. for inspection in a frontent such as SkyPortal). 

Such reactions take place in the T3 tier. A simple `T3HelloWorld` unit is used, but the  `react` method can be configured to do most other things. Sample T3 units react with TNS, Slack, Dropbox, SkyPortal and GCN.

Key steps of configuring the T3 procss comes through the `selection` directive, where we select tansients that produced the required target match, and the `execute` directive which regulates which T3 units are run. 

In [ ]:
# Test base
t3 = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "demo_SN09if",
    directives = [ {
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                't2_filter':  {
                    'unit': 'T2SNcosmoComp',
                    'match': {'target_match': True}
                }, 
            }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "DATAPOINT", "COMPOUND", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {
                            "project": {
                                "unit": "T3ChannelProjector",
                                "config": {
                                    "channel": "demo_SN09if"
                                }
                            },
                            "execute": [
                                {
                                    "unit": "T3HelloWorld",
                                    "config": {
                                        't2info_from' : ['T2SNcosmoComp', 'T2MultiMessMatch']
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)


In [ ]:
t3.run()

### Match to (multi-messenger) coincidence region

The most common way to work with multi-messenger data is to match optical data with some confidence region provided by another detector.

In [ ]:
controller??

In [ ]:
controller.stop()